<a href="https://colab.research.google.com/github/arham5siddiqui/Mitigating-Linkability-Attacks-through-Differential-Privacy-enabled-Neural-Network-Training/blob/main/User_Identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

Shuffling a Train file in chunks, in order include multiple Classes(multiple user data) in every chunk.

In [ ]:


input_file_path = "/content/drive/MyDrive/MSc Project/Formated_Data/Experiment_1/TrainTestDataset/Reduced_Combined_Train.csv"
output_file_path = "/content/drive/MyDrive/MSc Project/Formated_Data/Experiment_1/TrainTestDataset/Shuffled_Reduced_Combined_Train.csv"

chunk_size = 10000  # Choose a reasonable chunk size

# Initialize writing to CSV
first_one = True

# Read and shuffle each chunk, then append to new CSV
for chunk in pd.read_csv(input_file_path, chunksize=chunk_size):
    shuffled_chunk = chunk.sample(frac=1).reset_index(drop=True)

    if first_one:
        shuffled_chunk.to_csv(output_file_path, mode='w', index=False)
        first_one = False
    else:
        shuffled_chunk.to_csv(output_file_path, mode='a', header=False, index=False)


Using: all columns except labels 'userID' and 'videoID', as features.
First each classifier is trained,
and then tested, with the output printing accuracy table for each classifier.

In [ ]:


# Initialize classifiers
classifiers = {
    "SGD": SGDClassifier(),
    "GaussianNB": GaussianNB(),
    "RandomForest": RandomForestClassifier(),
    "DecisionTree": DecisionTreeClassifier(),
    "KNN": KNeighborsClassifier()
}

# Initialize a flag to indicate if each classifier has been fitted
fitted = {name: False for name in classifiers.keys()}

# Initialize empty dictionary to store accuracy
accuracy_dict = {'Classifier': [], 'Dataset': [], 'Accuracy': []}

# Initialize empty list to store train accuracy
train_accuracy_list = []

# Initialize final dictionary to store average accuracies
final_dict = {'Classifier': [], 'Dataset': [], 'Accuracy': [], 'Train Accuracy': []}

# Initialize the list of dataset types
dataset_types = ['Quaternion', 'Euler', 'Yaw']  # Adding actual dataset types

# Define chunk size
chunk_size = 10 ** 6  # Adjust based on your available memory
subset_size = 10 ** 4  # Size for training classifiers that don't support partial_fit

# Read a small portion of the file to get the unique classes
unique_classes = set()
for chunk in pd.read_csv("/content/drive/MyDrive/MSc Project/Formated_Data/Experiment_1/TrainTestDataset/Shuffled_Reduced_Combined_Train.csv", chunksize=chunk_size):
    unique_classes.update(chunk['userID'])
unique_classes = np.array(list(unique_classes))


# Function to train classifiers that support partial_fit
def train_partial_fit(chunk):
    X_train = chunk.drop(['userID', 'videoID'], axis=1)
    y_train = chunk['userID']
    if len(np.unique(y_train)) > 1:  # Check for multiple unique classes
        for name, clf in classifiers.items():
            if hasattr(clf, 'partial_fit'):
                clf.partial_fit(X_train, y_train, classes=unique_classes)
                fitted[name] = True  # Mark as fitted

# Function to test classifiers
def test_classifiers(chunk, dataset_type):
    X_test = chunk.drop(['userID', 'videoID'], axis=1)
    y_test = chunk['userID']
    for name, clf in classifiers.items():
        if fitted[name]:  # Only test if the classifier was fitted
            y_pred = clf.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_dict['Classifier'].append(name)
            accuracy_dict['Dataset'].append(dataset_type)
            accuracy_dict['Accuracy'].append(accuracy)


# Training loop for classifiers that support partial_fit
for chunk in pd.read_csv("/content/drive/MyDrive/MSc Project/Formated_Data/Experiment_1/TrainTestDataset/Shuffled_Reduced_Combined_Train.csv", chunksize=chunk_size):
    train_partial_fit(chunk)

# Fit classifiers that do not support partial_fit on a smaller subset
subset = pd.read_csv("/content/drive/MyDrive/MSc Project/Formated_Data/Experiment_1/TrainTestDataset/Shuffled_Reduced_Combined_Train.csv", nrows=subset_size)
X_subset = subset.drop(['userID', 'videoID'], axis=1)
y_subset = subset['userID']
for name, clf in classifiers.items():
    if not fitted[name]:
        clf.fit(X_subset, y_subset)
        fitted[name] = True
        train_accuracy = clf.score(X_subset, y_subset)
        train_accuracy_list.append((name, train_accuracy))

# Test the classifiers
for chunk in pd.read_csv("/content/drive/MyDrive/MSc Project/Formated_Data/Experiment_1/TrainTestDataset/Reduced_Combined_Test.csv", chunksize=chunk_size):
    test_classifiers(chunk, 'Test')

# Average the test accuracy and fill in the dictionary for DataFrame creation
for name in classifiers.keys():
    indices = [i for i, x in enumerate(accuracy_dict['Classifier']) if x == name]
    if indices:
        avg_accuracy = np.mean([accuracy_dict['Accuracy'][i] for i in indices])
        final_dict['Classifier'].append(name)
        final_dict['Dataset'].append('Test')  # This will be 'Test' for all classifiers in this case
        final_dict['Accuracy'].append(avg_accuracy)
        final_dict['Train Accuracy'].append(np.nan)  # Placeholder for train accuracy

# Incorporate the train_accuracy_list into the final_dict
for name, train_accuracy in train_accuracy_list:
    if name in final_dict['Classifier']:
        idx = final_dict['Classifier'].index(name)
        final_dict['Train Accuracy'][idx] = train_accuracy

# Save results to CSV
results_df = pd.DataFrame(final_dict)
results_df.to_csv("/content/drive/MyDrive/MSc Project/Formated_Data/Experiment_1/TrainTestDataset/results.csv", index=False)





In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Initialize classifiers
classifiers = {
    "SGD": SGDClassifier(),
    "GaussianNB": GaussianNB(),
    "RandomForest": RandomForestClassifier(),
    "DecisionTree": DecisionTreeClassifier(),
    "KNN": KNeighborsClassifier()
}

# Initialize a flag to indicate if each classifier has been fitted
fitted = {name: False for name in classifiers.keys()}

# Initialize empty dictionary to store accuracy
accuracy_dict = {'Classifier': [], 'Dataset': [], 'Accuracy': []}

# Initialize empty list to store train accuracy
train_accuracy_list = []

# Initialize final dictionary to store average accuracies
final_dict = {'Classifier': [], 'Dataset': [], 'Accuracy': [], 'Train Accuracy': []}

# Initialize the list of dataset types
dataset_types = ['Quaternion', 'Euler', 'Yaw']  # Adding actual dataset types

# Initialize running sum and count for training accuracy for classifiers that support partial_fit
train_accuracy_sum = {name: 0.0 for name in classifiers.keys()}
train_accuracy_count = {name: 0 for name in classifiers.keys()}

# Define chunk size
chunk_size = 10 ** 6  # Adjust based on your available memory
subset_size = 10 ** 4  # Size for training classifiers that don't support partial_fit

# Read a small portion of the file to get the unique classes
unique_classes = set()
for chunk in pd.read_csv("/content/drive/MyDrive/MSc Project/Formated_Data/Experiment_1/TrainTestDataset/Shuffled_Reduced_Combined_Train.csv", chunksize=chunk_size):
    unique_classes.update(chunk['userID'])
unique_classes = np.array(list(unique_classes))


def train_partial_fit(chunk):
    X_train = chunk.drop(['userID', 'videoID'], axis=1)
    y_train = chunk['userID']
    if len(np.unique(y_train)) > 1:
        for name, clf in classifiers.items():
            if hasattr(clf, 'partial_fit'):
                clf.partial_fit(X_train, y_train, classes=unique_classes)
                train_accuracy = clf.score(X_train, y_train)  # Calculate train accuracy for this chunk
                train_accuracy_sum[name] += train_accuracy * len(y_train)  # Update the running sum
                train_accuracy_count[name] += len(y_train)  # Update the count
                fitted[name] = True

# Function to test classifiers
def test_classifiers(chunk, dataset_type):
    X_test = chunk.drop(['userID', 'videoID'], axis=1)
    y_test = chunk['userID']
    for name, clf in classifiers.items():
        if fitted[name]:  # Only test if the classifier was fitted
            y_pred = clf.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_dict['Classifier'].append(name)
            accuracy_dict['Dataset'].append(dataset_type)
            accuracy_dict['Accuracy'].append(accuracy)


# Training loop for classifiers that support partial_fit
for chunk in pd.read_csv("/content/drive/MyDrive/MSc Project/Formated_Data/Experiment_1/TrainTestDataset/Shuffled_Reduced_Combined_Train.csv", chunksize=chunk_size):
    train_partial_fit(chunk)

# Fit classifiers that do not support partial_fit on a smaller subset
subset = pd.read_csv("/content/drive/MyDrive/MSc Project/Formated_Data/Experiment_1/TrainTestDataset/Shuffled_Reduced_Combined_Train.csv", nrows=subset_size)
X_subset = subset.drop(['userID', 'videoID'], axis=1)
y_subset = subset['userID']
for name, clf in classifiers.items():
    if not fitted[name]:
        clf.fit(X_subset, y_subset)
        fitted[name] = True
        train_accuracy = clf.score(X_subset, y_subset)
        train_accuracy_list.append((name, train_accuracy))

# Test the classifiers
for chunk in pd.read_csv("/content/drive/MyDrive/MSc Project/Formated_Data/Experiment_1/TrainTestDataset/Reduced_Combined_Test.csv", chunksize=chunk_size):
    test_classifiers(chunk, 'Test')


# Average the training accuracy for classifiers that support partial_fit
for name in classifiers.keys():
    if train_accuracy_count[name] > 0:
        avg_train_accuracy = train_accuracy_sum[name] / train_accuracy_count[name]
        final_dict['Train Accuracy'].append(avg_train_accuracy)
    else:
        final_dict['Train Accuracy'].append(np.nan)  # Placeholder for classifiers that were not fitted

print({key: len(value) for key, value in final_dict.items()})

# Save results to CSV
results_df = pd.DataFrame(final_dict)
results_df.to_csv("/content/drive/MyDrive/MSc Project/Formated_Data/Experiment_1/TrainTestDataset/results.csv", index=False)





/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:514: RuntimeWarning: divide by zero encountered in log
  jointi = np.log(self.class_prior_[i])
/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:514: RuntimeWarning: divide by zero encountered in log
  jointi = np.log(self.class_prior_[i])


{'Classifier': 0, 'Dataset': 0, 'Accuracy': 0, 'Train Accuracy': 5}


ValueError: ignored

In [ ]:
import pandas as pd
import random
import os

def shuffle_large_file(file_path, temp_path, chunk_size=10000):
    # Step 1: Sort the original file based on 'userID' and 'videoID'
    sorted_chunks = []
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        sorted_chunk = chunk.sort_values(by=['userID', 'videoID'])
        sorted_chunks.append(sorted_chunk)

    sorted_df = pd.concat(sorted_chunks, ignore_index=True)
    sorted_df.to_csv(temp_path, index=False)

    # Step 2: Shuffle within each chunk and save to temporary file
    shuffled_chunks = []
    for chunk in pd.read_csv(temp_path, chunksize=chunk_size):
        grouped = chunk.groupby(['userID', 'videoID'])
        groups = [group for name, group in grouped]
        random.shuffle(groups)
        shuffled_chunk = pd.concat(groups).reset_index(drop=True)
        shuffled_chunks.append(shuffled_chunk)

    # Step 3: Shuffle the chunks and write to final file
    random.shuffle(shuffled_chunks)
    final_df = pd.concat(shuffled_chunks, ignore_index=True)
    final_df.to_csv(file_path, index=False)

    # Remove the temporary file
    os.remove(temp_path)

# List of file paths to be shuffled
file_paths = [
    '/content/drive/MyDrive/MSc Project/Formated_Data/Experiment_1/TrainTestDataset/Reduced_Combined_Train.csv',
    # Add more file paths if needed
]

# Temporary file path
temp_path = '/content/drive/MyDrive/MSc Project/Formated_Data/Experiment_1/TrainTestDataset/temp_sorted.csv'

# Shuffle each file in chunks
for file_path in file_paths:
    shuffle_large_file(file_path, temp_path)


OSError: ignored